In [7]:
%load_ext sql
%sql sqlite:///flights.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


u'Connected: @flights.db'

Problem Set 1
=======

### Instructions / Notes:

**_Read these carefully_**


* You will need to install the `prettytable` module to run the scripts. (eg. `pip install --user prettytable`)
* Download `flights.db` from http://web.stanford.edu/class/cs145/files/flights.db and make sure it is in the same directory as this Jupyter notebook 
* Run the top cell above to load the database `flights.db`.
* You **may** create new Jupyter notebook cells to use for e.g. testing, debugging, exploring, etc.- this is encouraged in fact!
* When you see `In [*]:` to the left of the cell you are executing, this means that the code / query is _running_.
    * **If the cell is hanging- i.e. running for too long: to restart the SQL connection, you must restart the entire python kernel**
    * To restart the kernel using the menu bar: "Kernel >> Restart & Clear Output"), then re-execute the sql connection cell at the top
    * You will also need to restart the connection if you want to load a different version of the database file
* Remember:
    * `%sql [SQL]` is for _single line_ SQL queries
    * `%%sql 
    [SQL]` is for _multi line_ SQL queries
* Running `submit.py` will run the queries and print them. 
* We have provided correct output from our solution in `correct_output.txt`.
  * You can diff your output with the correct output by running `python sanity_check.py` or `python submit.py > my_output; diff my_output correct_output.txt`
   * **Your `submit.py` should match this output exactly.** This means:
       * the columns should have the **same names** as `correct_output.txt`
       * the columns should be in the **same order** as `correct_output.txt`

### Submission Instructions:
 * Do _NOT_ submit your iPython notebook.
 * Instead, copy each query into the provided `submit.py` script in the corresponding variable.
   * Do _not_ copy the `%sql` or `%%sql` commands, only the raw SQL.
   * We will autograde your queries on a database with identical schema but arbitrarily-chosen values. Therefore, do not hard-code any constants in your queries -- all of your queries should use the tables in the database.
 * **Follow the instructions in `submission_instructions.txt` to upload your answers to our autograders.**

_Have fun!_

Introduction: Travel Delays
------------------------

There's nothing I dislike more than travel delays -- how about you?

In fact, I'm always scheming new ways to avoid travel delays, and I just found an amazing dataset that will help me understand some of the causes and trade-offs when traveling.

I wonder if you can use SQL to help me!

----

Not surprisingly... you can!

In this homework, we'll use SQL to explore airline travel delays that occurred in July 2017.

To start, let's look at the primary relation in the database we've prepared for you:

In [8]:
%%sql
SELECT * 
FROM flight_delays 
LIMIT 1;

 * sqlite:///flights.db
Done.


year,quarter,month,day_of_month,day_of_week,fl_date,unique_carrier,airline_id,carrier,tail_num,fl_num,origin_airport_id,origin_airport_seq_id,origin_city_market_id,origin,origin_city_name,origin_state_abr,origin_state_fips,origin_state_nm,origin_wac,dest_airport_id,dest_airport_seq_id,dest_city_market_id,dest,dest_city_name,dest_state_abr,dest_state_fips,dest_state_nm,dest_wac,crs_dep_time,dep_time,dep_delay,dep_delay_new,dep_del15,dep_delay_group,dep_time_blk,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,arr_delay_new,arr_del15,arr_delay_group,arr_time_blk,cancelled,cancellation_code,diverted,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,unnamed: 55
2017,3,7,1,6,2017-07-01,AS,19930,AS,N559AS,1,11278,1127803,30852,DCA,"Washington, DC",VA,51,Virginia,38,14747,1474703,30559,SEA,"Seattle, WA",WA,53,Washington,93,800,750.0,-10.0,0.0,0.0,-1.0,0800-0859,17.0,807.0,1010.0,11.0,1053,1021.0,-32.0,0.0,0.0,-2.0,1000-1059,0.0,None,0.0,None,None,None,None,None,None


Cool, there are so many columns! How many rows are there?

In [9]:
%%sql
SELECT COUNT(*) AS num_rows
FROM flight_delays

 * sqlite:///flights.db
Done.


num_rows
509070


Wow, that's a lot of data! Good thing you don't have to answer all of my questions by hand...

You don't need to import more data into the database. However, you can find a description of each field online at [https://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=236](https://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=236).

We've pre-loaded a number of additional tables that will help you decode important fields like `airline_id`, `airport_id`, and `day_of_week` into human-readable form for the queries below.

Please use the following cell to explore these the `airlines` and `weekdays` tables:

In [11]:
%%sql
SELECT * FROM airlines LIMIT 1

 * sqlite:///flights.db
Done.


airline_id,airline_name
19031,Mackey International Inc.: MAC


Alright -- let's get started!

# Part I: SQL Queries

Query 1: How long are flights delayed on average? (10 points)
------------------------
Just to get a sense of the data, let's start with a simple query.

In the cell below, write a SQL query that returns the average arrival delay for the entire month of July 2017 (i.e., the whole dataset).

In [12]:
%%sql
SELECT AVG(arr_delay) as avg_delay FROM flight_delays
WHERE year = 2017 AND month = 7



 * sqlite:///flights.db
Done.


avg_delay
8.2951476705


Query 2: What was the worst flight delay? (10 points)
------------------------
Hmm, the average doesn't look too bad! What about the _worst_ delay?

In the cell below, write a SQL query that returns the maximum arrival delay for the entire month of July 2017 (i.e., the whole dataset).

In [13]:
%%sql
SELECT MAX(arr_delay) as max_delay FROM flight_delays
WHERE year = 2017 AND month = 7


 * sqlite:///flights.db
Done.


max_delay
1895.0


Query 3: What flight am I happiest I didn't take? (10 points)
------------------------
Yikes! What flight was so late?

In the cell below, write a SQL query that returns the carrier (i.e., `carrier`), flight number, origin city name, arrival city name, and flight date for the flight with the maximum arrival delay for the entire month of July 2017. Do not hard-code the arrival delay you found above. Hint: use a subquery.

In [14]:
%%sql
SELECT carrier, fl_num, origin_city_name, dest_city_name, fl_date
    FROM flight_delays
    WHERE arr_delay = (SELECT MAX(arr_delay) FROM flight_delays WHERE year = 2017 AND month = 7)


 * sqlite:///flights.db
Done.


carrier,fl_num,origin_city_name,dest_city_name,fl_date
AA,58,"Kona, HI","Los Angeles, CA",2017-07-26


Query 4: Which are the worst days to travel? (10 points)
------------------------
Since CS145 just started, I don't have time to head to Kona anytime soon. However, I'm headed out of town for a trip next week! What day is worst for booking my flight?

In the cell below, write a SQL query that returns the average arrival delay time for each day of the week, in descending order. The schema of your relation should be of the form (`weekday_name`, `average_delay`).

**Note: do _not_ report the weekday ID.** (Hint: look at the `weekdays` table and perform a join to obtain the weekday name.)

In [15]:
%%sql
SELECT weekday_name,sub.average_delay
FROM weekdays
INNER JOIN (SELECT AVG(arr_delay) AS average_delay,day_of_week
      FROM flight_delays
    GROUP BY day_of_week)sub
ON weekdays.weekday_id = sub.day_of_week
ORDER BY average_delay DESC

 * sqlite:///flights.db
Done.


weekday_name,average_delay
Friday,14.4520127056
Monday,10.5375015249
Thursday,8.47985564693
Wednesday,8.4561902339
Saturday,7.54455459234
Tuesday,4.63152453983
Sunday,4.21165978081


Query 5: Which airlines that fly out of SFO are delayed least?
------------------------
Now that I know which days to avoid, I'm curious which airline I should fly out of SFO. Since I haven't been told where I'm flying, please just compute the average for the airlines that fly from SFO.

In the cell below, write a SQL query that returns the average arrival delay time (across _all_ flights) for each carrier that flew out of SFO at least once in July 2017 (i.e., in the current dataset), in descending order.

**Note: do _not_ report the airlines ID.** (Hint: a subquery is helpful here; also, look at the `airlines` table and perform a join.)

In [16]:
%%sql

SELECT airline_name, avg_delay
FROM
(
    SELECT AVG(arr_delay) as avg_delay, airline_id
    FROM flight_delays
    GROUP BY airline_id
)sub1
INNER JOIN
(
    SELECT airline_id
    FROM flight_delays
    WHERE origin_city_name = 'San Francisco, CA' AND year = 2017 AND month  = 7 
    GROUP BY airline_id
)sub2
ON sub1.airline_id = sub2.airline_id
INNER JOIN
airlines
ON sub1.airline_id = airlines.airline_id
ORDER BY avg_delay DESC


 * sqlite:///flights.db
Done.


airline_name,avg_delay
JetBlue Airways: B6,21.4594984033
American Airlines Inc.: AA,11.1863163459
Frontier Airlines Inc.: F9,10.1060463167
SkyWest Airlines Inc.: OO,9.09980190548
Southwest Airlines Co.: WN,8.77770549059
United Air Lines Inc.: UA,6.73033304351
Virgin America: VX,6.06631734076
Delta Air Lines Inc.: DL,2.7830355107
Hawaiian Airlines Inc.: HA,-0.269880510919
Alaska Airlines Inc.: AS,-1.10981736509


Query 6: What proportion of airlines are regularly late?
------------------------
Yeesh, there are a lot of late flights! How many airlines are regularly late?

In the cell below, write a SQL query that returns the proportion of airlines (appearing in `flight_delays`) whose flights are on average at least 10 minutes late to arrive. Do not hard-code the total number of airlines, and make sure to use at least one `HAVING` clause in your SQL query.

Note: sqlite `COUNT(*)` returns integer types. Therefore, your query should likely contain at least one `SELECT CAST (COUNT(*) AS float)` or a clause like `COUNT(*)*1.0`.

In [17]:
%%sql
SELECT COUNT(t2. down)*1.0/COUNT(t1.up) as late_proportion
FROM(SELECT carrier as up FROM flight_delays GROUP BY carrier)t1
LEFT JOIN
(SELECT carrier as down FROM flight_delays GROUP BY carrier HAVING AVG(arr_delay)>=10)t2
on t1.up = t2. down



 * sqlite:///flights.db
Done.


late_proportion
0.333333333333


Query 7: How do late departures affect late arrivals?
------------------------
It sure looks like my plane is likely to be delayed. I'd like to know: if my plane is delayed in taking off, how will it affect my arrival time?

The [sample covariance](https://en.wikipedia.org/wiki/Covariance) provides a measure of the joint variability of two variables. The higher the covariance, the more the two variables behave similarly, and negative covariance indicates the variables indicate the variables tend to be inversely related. We can compute the sample covariance as:
$$
Cov(X,Y) = \frac{1}{n-1} \sum_{i=1}^n (x_i-\hat{x})(y_i-\hat{y})
$$
where $x_i$ denotes the $i$th sample of $X$, $y_i$ the $i$th sample of $Y$, and the mean of $X$ and $Y$ are denoted by $\bar{x}$ and $\bar{y}$.

In the cell below, write a single SQL query that computes the covariance between the departure delay time and the arrival delay time.

*Note: we could also compute a statistic like the [Pearson correlation coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) here, which provides a normalized measure (i.e., on a scale from -1 to 1) of how strongly two variables are related. However, sqlite doesn't natively support square roots (unlike commonly-used relational databases like PostgreSQL and MySQL!), so we're asking you to compute covariance instead.*

In [18]:
%%sql
SELECT SUM(arr*dep-arr*dd-dep*ad+ad*dd)/COUNT(*) as cov
FROM (SELECT arr_delay as arr, dep_delay as dep 
    FROM flight_delays
    --WHERE dep_delay > 0
)t1
CROSS JOIN(
    SELECT AVG(arr_delay) as ad, AVG(dep_delay) as dd
    FROM flight_delays
    --WHERE dep_delay > 0
)t2


 * sqlite:///flights.db
Done.


cov
2367.26611877


Query 8: It was a bad week...
------------------------
Which airlines had the largest absolute increase in average arrival delay in the last week of July (i.e., flights on or after July 24th) compared to the previous days (i.e. flights before July 24th)?

In the cell below, write a single SQL query that returns the airline name (_not_ ID) with the maximum absolute increase in average arrival delay between the first 23 days of the month and days 24-31. Report both the airline name and the absolute increase.

Note: due to [sqlite's handling of dates](http://www.sqlite.org/lang_datefunc.html), it may be easier to query using `day_of_month`.

Note 2: This is probably the hardest query of the assignment; break it down into subqueries that you can run one-by-one and build up your answer subquery by subquery.

Hint: You can compute two subqueries, one to compute the average arrival delay for flights on or after July 24th, and one to compute the average arrival delay for flights before July 24th, and then join the two to calculate the increase in delay. 

In [19]:
%%sql
select airline_name, MAX(sub2.after - sub1.before) as delay_increase

FROM (
    select AVG(arr_delay) as before, airline_id, day_of_month
    FROM flight_delays
    WHERE month = 7 AND day_of_month <= 23 
    GROUP BY carrier
    )sub1
INNER JOIN (
    select AVG(arr_delay) as after,airline_id
    FROM flight_delays
  WHERE month = 7 AND day_of_month > 23
   GROUP BY carrier
)sub2
ON sub1.airline_id = sub2.airline_id
INNER JOIN
airlines
ON sub1.airline_id = airlines.airline_id


 * sqlite:///flights.db
Done.


airline_name,delay_increase
Southwest Airlines Co.: WN,1.23652594144


Query 9: Of Hipsters and Anarchists
------------------------
I'm keen to visit both Portland (PDX) and Eugene (EUG), but I can't fit both into the same trip. To maximize my frequent flier mileage, I'd like to use the same flight for each. Which airlines fly both SFO -> PDX and SFO -> EUG?

In the cell below, write a single SQL query that returns the distinct airline names (_not_ ID, and with no duplicates) that flew both SFO -> PDX and SFO -> EUG in July 2017.

In [20]:
%%sql
select airline_name
FROM (
    select airline_id
    FROM flight_delays
    WHERE month = 7 AND dest = 'PDX' AND origin = 'SFO'
    GROUP BY carrier
    )sub1
INNER JOIN (
    select airline_id
    FROM flight_delays
    WHERE month = 7 AND dest = 'EUG' AND origin = 'SFO'
    GROUP BY carrier
)sub2
ON sub1.airline_id = sub2.airline_id
INNER JOIN
airlines
ON sub1.airline_id = airlines.airline_id

 * sqlite:///flights.db
Done.


airline_name
SkyWest Airlines Inc.: OO
United Air Lines Inc.: UA


Query 10: Decision Fatigue and Equidistance
------------------------
I'm flying back to Stanford from Chicago later this month, and I can fly out of either Midway (MDW) or O'Hare (ORD) and can fly into either San Francisco (SFO), San Jose (SJC), or Oakland (OAK). If this month is like July, which leg will have the shortest arrival delay for flights leaving Chicago after 2PM local time?

In the cell below, write a single SQL query that returns the average arrival delay of flights departing either MDW or ORD after 2PM local time (`crs_dep_time`) and arriving at one of SFO, SJC, or OAK. Group by departure and arrival airport and return results descending by arrival delay.

Note: the `crs_dep_time` field is an integer formatted as hhmm (e.g. 4:15pm is 1615)

In [21]:
%%sql
select origin, dest, AVG(arr_delay) AS avg_delay
FROM flight_delays
WHERE (origin = 'MDW' OR origin = 'ORD') AND (dest = 'SFO' OR dest = 'SJC' OR dest = 'OAK') AND (crs_dep_time > 1400)
GROUP BY origin, dest
ORDER BY avg_delay DESC

 * sqlite:///flights.db
Done.


origin,dest,avg_delay
MDW,SFO,15.1142857143
ORD,SFO,14.2155172414
MDW,SJC,10.6
ORD,SJC,7.31111111111
MDW,OAK,4.75806451613


## You're done! Now submit!
 * Refer to the top of this notebook for submission instructions.